In [1]:
# === base ===
from pathlib import Path
import os
import gc
import warnings

# === data ===
import numpy as np
import pandas as pd

# === metrics & splits ===
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score

# === models (tabular/meta) ===
from catboost import CatBoostClassifier, Pool
# import lightgbm as lgb

warnings.filterwarnings("ignore")

In [2]:
print("CWD:", Path().resolve())

CWD: C:\Users\Nolan\Documents\My dokuments\GoIT\Deep_learning\petfinder_project\kaggle_notebooks


In [3]:
comp_path = Path("../data/artifacts")

trained_path = Path("../data/raw/train.csv")

oof_cnn = comp_path / "cnn_oof.csv"
test_cnn = comp_path / "cnn_test.csv"

oof_nlp = comp_path / "nlp_oof.csv"
test_nlp = comp_path / "nlp_test.csv"

oof_tabular = comp_path / "tabular_oof.csv"
test_tabular = comp_path / "tabular_test.csv"

print("oof_cnn:", oof_cnn.exists())
print("test_cnn:", test_cnn.exists())
print("oof_nlp:", oof_nlp.exists())
print("test_nlp:", test_nlp.exists())
print("trained_path:", trained_path.exists())
print("oof_tabular:", oof_tabular.exists())
print("test_tabular:", test_tabular.exists())

oof_cnn: True
test_cnn: True
oof_nlp: True
test_nlp: True
trained_path: True
oof_tabular: True
test_tabular: True


In [4]:
df_oof_cnn = pd.read_csv(oof_cnn)
df_test_cnn = pd.read_csv(test_cnn)

df_oof_nlp = pd.read_csv(oof_nlp)
df_test_nlp = pd.read_csv(test_nlp)

df_oof_tabular = pd.read_csv(oof_tabular)
df_test_tabular = pd.read_csv(test_tabular)

In [5]:
# print("OOF_CNN")
# print(df_oof_cnn.head(), "\n")
# print("OOF_NLP")
# print(df_oof_nlp.head(), "\n")
print("TEST_CNN")
print(df_test_cnn.head(), "\n")
print("TEST_NLP")
print(df_test_nlp.head())
print("TEST_TABULAR")
print(df_test_tabular.head(), "\n")
print("OOF_TABULAR")
print(df_oof_tabular.head())

TEST_CNN
       PetID    pred_0    pred_1    pred_2    pred_3
0  0008c5398  0.141417  0.244455  0.334568  0.279561
1  004c2f355  0.611917  0.256312  0.087698  0.044073
2  00553ae55  0.702013  0.251954  0.032555  0.013477
3  00eca0391  0.119653  0.450073  0.155823  0.274450
4  010a59236  0.088664  0.228364  0.232231  0.450740 

TEST_NLP
       PetID  nlp_proba_1  nlp_proba_2  nlp_proba_3  nlp_proba_4  nlp_pred
0  6697a7f62     0.186414     0.523980     0.277891     0.011716         2
1  23b64fe21     0.192035     0.273034     0.241004     0.293928         4
2  41e824cbe     0.079121     0.136374     0.418947     0.365558         3
3  6c3d7237b     0.118374     0.193876     0.083084     0.604667         4
4  97b0b5d92     0.244353     0.526338     0.110799     0.118510         2
TEST_TABULAR
       PetID    pred_0    pred_1    pred_2    pred_3
0  6697a7f62  0.211756  0.356579  0.178026  0.253638
1  23b64fe21  0.317650  0.234815  0.184570  0.262964
2  41e824cbe  0.122476  0.260343  0.2525

In [6]:
train_df = pd.read_csv(trained_path)
train_df.head()

,PetID,Description,AdoptionSpeed
0,d3b4f29f8,Mayleen and Flo are two lovely adorable sister...,2
1,e9dc82251,A total of 5 beautiful Tabbys available for ad...,2
2,8111f6d4a,Two-and-a-half month old girl. Very manja and ...,2
3,693a90fda,Neil is a healthy and active ~2-month-old fema...,2
4,9d08c85ef,Gray kitten available for adoption in sungai p...,2


In [7]:
df_oof_cnn = df_oof_cnn.drop(columns=["fold"])
df_oof_cnn = df_oof_cnn.rename(columns={"pred_0": "cnn_1", "pred_1": "cnn_2", "pred_2": "cnn_3", "pred_3": "cnn_4"})
df_oof_cnn = df_oof_cnn.merge(train_df[["PetID", "AdoptionSpeed"]], on="PetID", how="left")
df_oof_cnn.head()


,PetID,cnn_1,cnn_2,cnn_3,cnn_4,AdoptionSpeed
0,002230dea,0.381681,0.329058,0.169260,0.120001,1
1,0045ed62a,0.459765,0.391230,0.113191,0.035814,3
2,0058586f1,0.054000,0.180293,0.334604,0.431102,4
3,0063f83c9,0.302021,0.323644,0.202464,0.171871,1
4,007ffeccd,0.151347,0.521382,0.252788,0.074483,2


In [8]:
df_oof_nlp = df_oof_nlp.drop(columns=["nlp_pred"])
df_oof_nlp = df_oof_nlp.rename(columns={"nlp_proba_1": "nlp_1", "nlp_proba_2": "nlp_2", "nlp_proba_3": "nlp_3", "nlp_proba_4": "nlp_4"})
df_oof_nlp = df_oof_nlp.merge(train_df[["PetID", "AdoptionSpeed"]], on="PetID", how="left")
df_oof_nlp.head()


,PetID,nlp_1,nlp_2,nlp_3,nlp_4,AdoptionSpeed
0,d3b4f29f8,0.046295,0.255541,0.572760,0.125404,2
1,e9dc82251,0.031043,0.137292,0.757052,0.074613,2
2,8111f6d4a,0.267836,0.447975,0.159992,0.124198,2
3,693a90fda,0.004819,0.990378,0.004433,0.000371,2
4,9d08c85ef,0.409939,0.294647,0.139724,0.155689,2


In [9]:
df_test_cnn = df_test_cnn.rename(columns={"pred_0": "cnn_1", 
                                          "pred_1": "cnn_2", 
                                          "pred_2": "cnn_3", 
                                          "pred_3": "cnn_4"}
                                )
df_test_cnn.head()

,PetID,cnn_1,cnn_2,cnn_3,cnn_4
0,0008c5398,0.141417,0.244455,0.334568,0.279561
1,004c2f355,0.611917,0.256312,0.087698,0.044073
2,00553ae55,0.702013,0.251954,0.032555,0.013477
3,00eca0391,0.119653,0.450073,0.155823,0.274450
4,010a59236,0.088664,0.228364,0.232231,0.450740


In [10]:
df_test_nlp = df_test_nlp.drop(columns=["nlp_pred"])
df_test_nlp = df_test_nlp.rename(columns={"nlp_proba_1": "nlp_1",
                                          "nlp_proba_2": "nlp_2",
                                          "nlp_proba_3": "nlp_3",
                                          "nlp_proba_4": "nlp_4"})
df_test_nlp.head()

,PetID,nlp_1,nlp_2,nlp_3,nlp_4
0,6697a7f62,0.186414,0.523980,0.277891,0.011716
1,23b64fe21,0.192035,0.273034,0.241004,0.293928
2,41e824cbe,0.079121,0.136374,0.418947,0.365558
3,6c3d7237b,0.118374,0.193876,0.083084,0.604667
4,97b0b5d92,0.244353,0.526338,0.110799,0.118510


In [11]:
df_oof_tabular = df_oof_tabular.rename(columns={"pred_0": "tab_1",
                                                "pred_1": "tab_2",
                                                "pred_2": "tab_3",
                                                "pred_3": "tab_4"})
df_oof_tabular = df_oof_tabular.merge(train_df[["PetID", "AdoptionSpeed"]], on="PetID", how="left")
df_oof_tabular.head()

,PetID,tab_1,tab_2,tab_3,tab_4,AdoptionSpeed
0,d3b4f29f8,0.142903,0.372078,0.303965,0.181055,2
1,e9dc82251,0.237147,0.320429,0.236537,0.205887,2
2,8111f6d4a,0.214448,0.240812,0.174104,0.370636,2
3,693a90fda,0.185045,0.303221,0.178406,0.333328,2
4,9d08c85ef,0.177462,0.299493,0.225220,0.297825,2


In [12]:
df_test_tabular = df_test_tabular.rename(columns={"pred_0": "tab_1",
                                                "pred_1": "tab_2",
                                                "pred_2": "tab_3",
                                                "pred_3": "tab_4"})
df_test_tabular.head()

,PetID,tab_1,tab_2,tab_3,tab_4
0,6697a7f62,0.211756,0.356579,0.178026,0.253638
1,23b64fe21,0.317650,0.234815,0.184570,0.262964
2,41e824cbe,0.122476,0.260343,0.252516,0.364666
3,6c3d7237b,0.134640,0.136507,0.146309,0.582543
4,97b0b5d92,0.174015,0.349569,0.127205,0.349212


In [13]:
df_train = df_oof_cnn.merge(df_oof_nlp, on=["PetID", "AdoptionSpeed"], how="left")
df_train.head()

,PetID,cnn_1,cnn_2,cnn_3,cnn_4,AdoptionSpeed,nlp_1,nlp_2,nlp_3,nlp_4
0,002230dea,0.381681,0.329058,0.169260,0.120001,1,0.480694,0.465669,0.041126,0.012511
1,0045ed62a,0.459765,0.391230,0.113191,0.035814,3,0.041187,0.844593,0.099301,0.014919
2,0058586f1,0.054000,0.180293,0.334604,0.431102,4,0.041653,0.098496,0.153555,0.706296
3,0063f83c9,0.302021,0.323644,0.202464,0.171871,1,0.335770,0.594462,0.056162,0.013606
4,007ffeccd,0.151347,0.521382,0.252788,0.074483,2,0.221223,0.743146,0.032318,0.003313


In [14]:
df_train = df_train.merge(df_oof_tabular, on=["PetID", "AdoptionSpeed"], how="left")
df_train.head()

,PetID,cnn_1,cnn_2,cnn_3,cnn_4,AdoptionSpeed,nlp_1,nlp_2,nlp_3,nlp_4,tab_1,tab_2,tab_3,tab_4
0,002230dea,0.381681,0.329058,0.169260,0.120001,1,0.480694,0.465669,0.041126,0.012511,0.265826,0.312812,0.193766,0.227595
1,0045ed62a,0.459765,0.391230,0.113191,0.035814,3,0.041187,0.844593,0.099301,0.014919,0.251620,0.394734,0.171302,0.182343
2,0058586f1,0.054000,0.180293,0.334604,0.431102,4,0.041653,0.098496,0.153555,0.706296,0.185216,0.177358,0.316185,0.321241
3,0063f83c9,0.302021,0.323644,0.202464,0.171871,1,0.335770,0.594462,0.056162,0.013606,0.259199,0.287483,0.168175,0.285143
4,007ffeccd,0.151347,0.521382,0.252788,0.074483,2,0.221223,0.743146,0.032318,0.003313,0.172659,0.313121,0.252466,0.261754


In [15]:
df_test = df_test_cnn.merge(df_test_nlp, on=["PetID"], how="left")
df_test.head()

,PetID,cnn_1,cnn_2,cnn_3,cnn_4,nlp_1,nlp_2,nlp_3,nlp_4
0,0008c5398,0.141417,0.244455,0.334568,0.279561,0.230213,0.334268,0.150466,0.285053
1,004c2f355,0.611917,0.256312,0.087698,0.044073,0.152962,0.407846,0.169247,0.269944
2,00553ae55,0.702013,0.251954,0.032555,0.013477,0.635695,0.191872,0.044674,0.127759
3,00eca0391,0.119653,0.450073,0.155823,0.274450,0.140418,0.385107,0.312610,0.161866
4,010a59236,0.088664,0.228364,0.232231,0.450740,0.211265,0.423319,0.244340,0.121076


In [16]:
df_test = df_test.merge(df_test_tabular, on=["PetID"], how="left")
df_test.head()

,PetID,cnn_1,cnn_2,cnn_3,cnn_4,nlp_1,nlp_2,nlp_3,nlp_4,tab_1,tab_2,tab_3,tab_4
0,0008c5398,0.141417,0.244455,0.334568,0.279561,0.230213,0.334268,0.150466,0.285053,0.154449,0.242226,0.167672,0.435653
1,004c2f355,0.611917,0.256312,0.087698,0.044073,0.152962,0.407846,0.169247,0.269944,0.173965,0.249270,0.170279,0.406486
2,00553ae55,0.702013,0.251954,0.032555,0.013477,0.635695,0.191872,0.044674,0.127759,0.238279,0.241819,0.115603,0.404299
3,00eca0391,0.119653,0.450073,0.155823,0.274450,0.140418,0.385107,0.312610,0.161866,0.090357,0.235640,0.254762,0.419240
4,010a59236,0.088664,0.228364,0.232231,0.450740,0.211265,0.423319,0.244340,0.121076,0.186168,0.277844,0.191013,0.344974


In [17]:
assert df_train.shape[0] == 6431
assert df_train["PetID"].nunique() == 6431
assert df_train["AdoptionSpeed"].notna().all()

In [18]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1891 entries, 0 to 1890
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PetID   1891 non-null   object 
 1   cnn_1   1891 non-null   float64
 2   cnn_2   1891 non-null   float64
 3   cnn_3   1891 non-null   float64
 4   cnn_4   1891 non-null   float64
 5   nlp_1   1891 non-null   float64
 6   nlp_2   1891 non-null   float64
 7   nlp_3   1891 non-null   float64
 8   nlp_4   1891 non-null   float64
 9   tab_1   1891 non-null   float64
 10  tab_2   1891 non-null   float64
 11  tab_3   1891 non-null   float64
 12  tab_4   1891 non-null   float64
dtypes: float64(12), object(1)
memory usage: 192.2+ KB


In [19]:
sub = pd.read_csv("submission.csv")
print("submission rows:", len(sub))
print(sub.head())
print(sub.tail())

submission rows: 1887
       PetID  AdoptionSpeed
0  6697a7f62              1
1  23b64fe21              2
2  41e824cbe              4
3  6c3d7237b              4
4  97b0b5d92              1
          PetID  AdoptionSpeed
1882  986e26eeb              4
1883  9b2316d19              4
1884  c60193e34              2
1885  4f7a70728              4
1886  9e758c0b0              1


In [20]:
print("df_test duplicates:", df_test["PetID"].duplicated().sum())
print("sub duplicates:", sub["PetID"].duplicated().sum())

df_test duplicates: 0
sub duplicates: 0


In [21]:
print("df_test_cnn dup:", df_test_cnn["PetID"].duplicated().sum())
print("df_test_nlp dup:", df_test_nlp["PetID"].duplicated().sum())

df_test_cnn dup: 0
df_test_nlp dup: 0


In [22]:
X_train = df_train.drop(columns=["PetID", "AdoptionSpeed"])
y_train = df_train["AdoptionSpeed"] - 1

X_test = df_test.drop(columns=["PetID"])


In [23]:
FEATURES = [c for c in df_train.columns if c.startswith("cnn_") or c.startswith("nlp_")]
assert len(FEATURES) == 8, f"Expected 8 features, got {len(FEATURES)}: {FEATURES}"

X = df_train[FEATURES].copy()
y = df_train["AdoptionSpeed"].astype(int).values  # 1..4
y0 = y - 1  # -> 0..3 (for CatBoost)

X_test = df_test[FEATURES].copy()

# -------------------------
# 2) CV OOF for sanity + final fit
# -------------------------
SEED = 42
N_SPLITS = 5

oof_pred0 = np.zeros(len(df_train), dtype=int)
oof_proba = np.zeros((len(df_train), 4), dtype=np.float32)

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

params = dict(
    loss_function="MultiClass",
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3.0,          # (weight decay analogue)
    random_strength=1.0,
    bootstrap_type="Bayesian",
    bagging_temperature=0.5,
    eval_metric="WKappa",
    auto_class_weights="Balanced",
    task_type="CPU",
    random_seed=SEED,
    verbose=False,
)
fold_qwks = []
for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y0), 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y0[tr_idx], y0[va_idx]

    train_pool = Pool(X_tr, y_tr)
    valid_pool = Pool(X_va, y_va)

    model = CatBoostClassifier(**params)
    model.fit(
        train_pool,
        eval_set=valid_pool,
        use_best_model=True,
        early_stopping_rounds=200,
        verbose=200,
    )

    proba_va = model.predict_proba(X_va)
    pred_va0 = np.argmax(proba_va, axis=1)

    oof_pred0[va_idx] = pred_va0
    oof_proba[va_idx] = proba_va

    qwk_fold = cohen_kappa_score((y_va + 1), (pred_va0 + 1), weights="quadratic")
    fold_qwks.append(qwk_fold)
    print(f"Fold {fold}: QWK = {qwk_fold:.4f}")

print("\n===== QWK SUMMARY =====")
if len(fold_qwks) == 0:
    print("fold_qwks is EMPTY -> CV loop didn't run or crashed before append().")
else:
    print("fold qwks:", [round(x, 4) for x in fold_qwks])
    print(f"mean QWK: {np.mean(fold_qwks):.4f}")
    print(f"best QWK: {np.max(fold_qwks):.4f} (fold {int(np.argmax(fold_qwks)) + 1})")
    print(f"worst QWK: {np.min(fold_qwks):.4f} (fold {int(np.argmin(fold_qwks)) + 1})")

# OOF QWK по всіх рядках (правильно в 1..4)
oof_pred_1to4 = oof_pred0 + 1
y_1to4 = y0 + 1
overall_qwk = cohen_kappa_score(y_1to4, oof_pred_1to4, weights="quadratic")
print(f"OVERALL OOF QWK: {overall_qwk:.4f}")

qwk_oof = cohen_kappa_score(y, (oof_pred0 + 1), weights="quadratic")
print(f"\nOOF QWK (CatBoost stacking): {qwk_oof:.4f}")


# Save OOF meta features (optional, but useful)
df_cb_oof = pd.DataFrame({
    "PetID": df_train["PetID"].values,
    "cb_1": oof_proba[:, 0],
    "cb_2": oof_proba[:, 1],
    "cb_3": oof_proba[:, 2],
    "cb_4": oof_proba[:, 3],
})
df_cb_oof.to_csv("cb_oof.csv", index=False)
print("saved: cb_oof.csv")

# -------------------------
# 3) Train final model on ALL data and predict TEST
# -------------------------
final_model = CatBoostClassifier(**params)
final_model.fit(Pool(X, y0), verbose=200)

test_proba = final_model.predict_proba(X_test)
test_pred = np.argmax(test_proba, axis=1) + 1  # back to 1..4

# Save test meta features (optional)
df_cb_test = pd.DataFrame({
    "PetID": df_test["PetID"].values,
    "cb_1": test_proba[:, 0],
    "cb_2": test_proba[:, 1],
    "cb_3": test_proba[:, 2],
    "cb_4": test_proba[:, 3],
})
df_cb_test.to_csv("cb_test.csv", index=False)
print("saved: cb_test.csv")

# Final submission
submission = pd.DataFrame({
    "PetID": df_test["PetID"].values,
    "AdoptionSpeed": test_pred.astype(int),
})
submission.to_csv("submission.csv", index=False)
print("saved: submission.csv")

0:	learn: 0.5154404	test: 0.4733580	best: 0.4733580 (0)	total: 100ms	remaining: 3m 20s
200:	learn: 0.5561084	test: 0.5046707	best: 0.5197934 (5)	total: 1.13s	remaining: 10.1s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.5197934042
bestIteration = 5

Shrink model to first 6 iterations.
Fold 1: QWK = 0.5334
0:	learn: 0.4873285	test: 0.4676494	best: 0.4676494 (0)	total: 5.85ms	remaining: 11.7s
200:	learn: 0.5559155	test: 0.5003463	best: 0.5115282 (81)	total: 1.06s	remaining: 9.53s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.5115281774
bestIteration = 81

Shrink model to first 82 iterations.
Fold 2: QWK = 0.5222
0:	learn: 0.5118495	test: 0.4933445	best: 0.4933445 (0)	total: 5.23ms	remaining: 10.5s
200:	learn: 0.5524287	test: 0.5188510	best: 0.5188510 (200)	total: 1.03s	remaining: 9.25s
400:	learn: 0.5715904	test: 0.5256227	best: 0.5261527 (395)	total: 2.14s	remaining: 8.54s
600:	learn: 0.5995377	test: 0.5187804	best: 0.5276899 (414)	total: 3

In [24]:
print("fold qwks:", [round(x, 4) for x in fold_qwks])
print(f"mean QWK: {np.mean(fold_qwks):.4f}")
print(f"best QWK: {np.max(fold_qwks):.4f} (fold {int(np.argmax(fold_qwks)) + 1})")
print(f"worst QWK: {np.min(fold_qwks):.4f} (fold {int(np.argmin(fold_qwks)) + 1})")

fold qwks: [0.5334, 0.5222, 0.5345, 0.5672, 0.5844]
mean QWK: 0.5484
best QWK: 0.5844 (fold 5)
worst QWK: 0.5222 (fold 2)
